<a href="https://colab.research.google.com/github/HamsikaSateesh/simple_ml_algorithms/blob/main/sms_spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.2 MB/s eta 0:00:00


In [6]:
from datasets import load_dataset

ds = load_dataset("ucirvine/sms_spam")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/359k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5574 [00:00<?, ? examples/s]

In [7]:
ds

DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 5574
    })
})

In [9]:
print(ds['train'][:5]) #1 : spam

{'sms': ['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...\n', 'Ok lar... Joking wif u oni...\n', "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's\n", 'U dun say so early hor... U c already then say...\n', "Nah I don't think he goes to usf, he lives around here though\n"], 'label': [0, 0, 1, 0, 0]}


In [23]:
import pandas as pd
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [12]:
df = pd.DataFrame(ds['train'])

In [14]:
df.head()

,sms,label
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...\n,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [18]:
#remove special characters
df['text_clean'] = df['sms'].str.lower().apply(lambda x: re.sub(r'[^\w\s]', '', x))

In [19]:
df['text_clean'].head()

,text_clean
0,go until jurong point crazy available only in ...
1,ok lar joking wif u oni\n
2,free entry in 2 a wkly comp to win fa cup fina...
3,u dun say so early hor u c already then say\n
4,nah i dont think he goes to usf he lives aroun...


In [20]:
df['text_clean'] = df['text_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in ENGLISH_STOP_WORDS]))

In [21]:
df['text_clean'].head()

,text_clean
0,jurong point crazy available bugis n great wor...
1,ok lar joking wif u oni
2,free entry 2 wkly comp win fa cup final tkts 2...
3,u dun say early hor u c say
4,nah dont think goes usf lives


In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['text_clean'], df['label'], test_size=0.2, random_state=42)

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [33]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer()
X_train_tfidf = tfidf.fit_transform(X_train_vec)
X_test_tfidf = tfidf.transform(X_test_vec)

In [60]:
#using naive bayes
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

y_pred = model.predict(X_test_tfidf)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


0.9713004484304932
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       954
           1       1.00      0.80      0.89       161

    accuracy                           0.97      1115
   macro avg       0.98      0.90      0.94      1115
weighted avg       0.97      0.97      0.97      1115



In [61]:
from sklearn.feature_extraction.text import CountVectorizer
test_message = ["Can you meet me tomorrow morning at 8am in front of the coffee shop?"]

test_message_clean = [" ".join(re.sub(r'[^\w\s]', '', msg.lower()).split()) for msg in test_message]

test_vector = vectorizer.transform(test_message_clean)

print(model.predict(test_vector))


[0]


In [62]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Train model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Predict
y_pred = model.predict(X_test_tfidf)

# Evaluate
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print(classification_report(y_test, y_pred))


Accuracy: 0.96
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       954
           1       0.98      0.74      0.84       161

    accuracy                           0.96      1115
   macro avg       0.97      0.87      0.91      1115
weighted avg       0.96      0.96      0.96      1115



In [63]:
from sklearn.feature_extraction.text import CountVectorizer
test_message = ["Congratulations! You've won a free iPhone! Call now!"]

test_message_clean = [" ".join(re.sub(r'[^\w\s]', '', msg.lower()).split()) for msg in test_message]

test_vector = vectorizer.transform(test_message_clean)

print(model.predict(test_vector))

[1]


In [64]:
#using nueral network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train_tfidf.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train_tfidf.toarray(), y_train, epochs=5, batch_size=32, validation_split=0.2)

loss, accuracy = model.evaluate(X_test_tfidf.toarray(), y_test)
print(f"Test Accuracy: {accuracy:.2f}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.8448 - loss: 0.5028 - val_accuracy: 0.9058 - val_loss: 0.1912
Epoch 2/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9614 - loss: 0.1152 - val_accuracy: 0.9731 - val_loss: 0.0829
Epoch 3/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9946 - loss: 0.0242 - val_accuracy: 0.9765 - val_loss: 0.0896
Epoch 4/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9993 - loss: 0.0064 - val_accuracy: 0.9731 - val_loss: 0.1101
Epoch 5/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9997 - loss: 0.0021 - val_accuracy: 0.9765 - val_loss: 0.1087
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9851 - loss: 0.0638
Test Accuracy: 0.98


In [51]:
from sklearn.feature_extraction.text import CountVectorizer
test_message = ["Congratulations! You've won a free iPhone! Call now!"]

test_message_clean = [" ".join(re.sub(r'[^\w\s]', '', msg.lower()).split()) for msg in test_message]

test_vector = vectorizer.transform(test_message_clean)

print(model.predict(test_vector))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
[[0.9999226]]


In [68]:
from sklearn.feature_extraction.text import CountVectorizer
test_message = ["Can you meet me tomorrow morning at 8am in front of the coffee shop?"]

test_message_clean = [" ".join(re.sub(r'[^\w\s]', '', msg.lower()).split()) for msg in test_message]

test_vector = vectorizer.transform(test_message_clean)

print((model.predict(test_vector))*100000000)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
[[0.68571784]]
